In [42]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_ta as ta
from math import *
from termcolor import colored as cl

plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (20,10)

In [31]:
df = yf.Ticker('INFY')
df = df.history(period='6mo')[['Open', 'High', 'Low', 'Close']]

In [32]:
df.tail()

,Open,High,Low,Close
Date,,,,
2023-08-29 00:00:00-04:00,17.280001,17.4300,17.260000,17.420000
2023-08-30 00:00:00-04:00,17.469999,17.5900,17.450001,17.549999
2023-08-31 00:00:00-04:00,17.500000,17.5700,17.360001,17.370001
2023-09-01 00:00:00-04:00,17.520000,17.5800,17.450001,17.520000
2023-09-05 00:00:00-04:00,17.820000,17.9524,17.760000,17.770000


In [33]:
print(df)

                                Open       High        Low      Close
Date                                                                 
2023-03-06 00:00:00-05:00  18.095238  18.095238  17.976839  18.016306
2023-03-07 00:00:00-05:00  18.006439  18.075505  17.878175  17.976839
2023-03-08 00:00:00-05:00  17.986704  17.986704  17.828840  17.917639
2023-03-09 00:00:00-05:00  17.730175  17.848574  17.483511  17.483511
2023-03-10 00:00:00-05:00  17.542712  17.710443  17.394712  17.444046
...                              ...        ...        ...        ...
2023-08-29 00:00:00-04:00  17.280001  17.430000  17.260000  17.420000
2023-08-30 00:00:00-04:00  17.469999  17.590000  17.450001  17.549999
2023-08-31 00:00:00-04:00  17.500000  17.570000  17.360001  17.370001
2023-09-01 00:00:00-04:00  17.520000  17.580000  17.450001  17.520000
2023-09-05 00:00:00-04:00  17.820000  17.952400  17.760000  17.770000

[127 rows x 4 columns]


In [34]:
def get_stoch_osc(high, low, Close, k_lookback, d_lookback):
    lowest_low = low.rolling(k_lookback).min()
    highest_high = high.rolling(k_lookback).max()
    k_line = ((Close - lowest_low) / (highest_high - lowest_low)) * 100
    d_line = k_line.rolling(d_lookback).mean()
    return k_line, d_line
df['%k'], df['%d'] = get_stoch_osc(df['High'], df['Low'], df['Close'], 14, 3)
df.tail()


,Open,High,Low,Close,%k,%d
Date,,,,,,
2023-08-29 00:00:00-04:00,17.280001,17.4300,17.260000,17.420000,98.850550,92.992928
2023-08-30 00:00:00-04:00,17.469999,17.5900,17.450001,17.549999,96.116419,95.544558
2023-08-31 00:00:00-04:00,17.500000,17.5700,17.360001,17.370001,77.777842,90.914937
2023-09-01 00:00:00-04:00,17.520000,17.5800,17.450001,17.520000,92.929322,88.941194
2023-09-05 00:00:00-04:00,17.820000,17.9524,17.760000,17.770000,86.512883,85.740016


In [35]:
def get_macd(price, slow, fast, smooth):
    exp1 = price.ewm(span = fast, adjust = False).mean()
    exp2 = price.ewm(span = slow, adjust = False).mean()
    macd = pd.DataFrame(exp1 - exp2).rename(columns = {'Close':'macd'})
    signal = pd.DataFrame(macd.ewm(span = smooth, adjust = False).mean()).rename(columns = {'macd':'signal'})
    hist = pd.DataFrame(macd['macd'] - signal['signal']).rename(columns = {0:'hist'})
    return macd, signal, hist
    
df['macd'] = get_macd(df['Close'], 26, 12, 9)[0]
df['macd_signal'] = get_macd(df['Close'], 26, 12, 9)[1]
df['macd_hist'] = get_macd(df['Close'], 26, 12, 9)[2]
df = df.dropna()
df.tail()

,Open,High,Low,Close,%k,%d,macd,macd_signal,macd_hist
Date,,,,,,,,,
2023-08-29 00:00:00-04:00,17.280001,17.4300,17.260000,17.420000,98.850550,92.992928,0.211314,0.181259,0.030056
2023-08-30 00:00:00-04:00,17.469999,17.5900,17.450001,17.549999,96.116419,95.544558,0.230803,0.191168,0.039635
2023-08-31 00:00:00-04:00,17.500000,17.5700,17.360001,17.370001,77.777842,90.914937,0.229083,0.198751,0.030332
2023-09-01 00:00:00-04:00,17.520000,17.5800,17.450001,17.520000,92.929322,88.941194,0.237090,0.206419,0.030672
2023-09-05 00:00:00-04:00,17.820000,17.9524,17.760000,17.770000,86.512883,85.740016,0.260605,0.217256,0.043349


In [18]:
def implement_stoch_macd_strategy(prices, k, d, macd, macd_signal):    
    buy_price = []
    sell_price = []
    stoch_macd_signal = []
    signal = 0

    for i in range(len(prices)):
        if k[i] < 30 and d[i] < 30 and macd[i] < -2 and macd_signal[i] < -2:
            if signal != 1:
                buy_price.append(prices[i])
                sell_price.append(np.nan)
                signal = 1
                stoch_macd_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                stoch_macd_signal.append(0)
                
        elif k[i] > 70 and d[i] > 70 and macd[i] > 2 and macd_signal[i] > 2:
            if signal != -1 and signal != 0:
                buy_price.append(np.nan)
                sell_price.append(prices[i])
                signal = -1
                stoch_macd_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                stoch_macd_signal.append(0)
        
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            stoch_macd_signal.append(0)
            
    return buy_price, sell_price, stoch_macd_signal
            
buy_price, sell_price, stoch_macd_signal = implement_stoch_macd_strategy(df['Close'], df['%k'], df['%d'], df['macd'], df['macd_signal'])

/tmp/ipykernel_22120/1134090288.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if k[i] < 30 and d[i] < 30 and macd[i] < -2 and macd_signal[i] < -2:
/tmp/ipykernel_22120/1134090288.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif k[i] > 70 and d[i] > 70 and macd[i] > 2 and macd_signal[i] > 2:


In [49]:
position = []
for i in range(len(stoch_macd_signal)):
    if stoch_macd_signal[i] > 1:
        position.append(0)
    else:
        position.append(1)
        
for i in range(len(df['Close'])):
    if stoch_macd_signal[i] == 1:
        position[i] = 1
    elif stoch_macd_signal[i] == -1:
        position[i] = 0
    else:
        position[i] = position[i-1]
        
Close_price = df['Close']
k_line = df['%k']
d_line = df['%d']
macd_line = df['macd']
signal_line = df['macd_signal']
stoch_macd_signal = pd.DataFrame(stoch_macd_signal).rename(columns = {0:'stoch_macd_signal'}).set_index(df.index)
position = pd.DataFrame(position).rename(columns = {0:'stoch_macd_position'}).set_index(df.index)

frames = [Close_price, k_line, d_line, macd_line, signal_line, stoch_macd_signal, position]
strategy = pd.concat(frames, join = 'inner', axis = 1)

strategy

KeyError: 0

In [50]:

df_ret = pd.DataFrame(np.diff(df['Close'])).rename(columns = {0:'returns'})
stoch_macd_strategy_ret = []

for i in range(len(df_ret)):
    try:
        returns = df_ret['returns'][i] * strategy['stoch_macd_position'][i]
        stoch_macd_strategy_ret.append(returns)
    except:
        pass
    
stoch_macd_strategy_ret_df = pd.DataFrame(stoch_macd_strategy_ret).rename(columns = {0:'stoch_macd_returns'})

investment_value = 100000
number_of_stocks = floor(investment_value / df['Close'][0])
stoch_macd_investment_ret = []

for i in range(len(stoch_macd_strategy_ret_df['stoch_macd_returns'])):
    returns = number_of_stocks * stoch_macd_strategy_ret_df['stoch_macd_returns'][i]
    stoch_macd_investment_ret.append(returns)

stoch_macd_investment_ret_df = pd.DataFrame(stoch_macd_investment_ret).rename(columns = {0:'investment_returns'})
total_investment_ret = round(sum(stoch_macd_investment_ret_df['investment_returns']), 2)
profit_percentage = floor((total_investment_ret / investment_value) * 100)
print(cl('Profit gained from the STOCH MACD strategy by investing $100k in Infosys : {}'.format(total_investment_ret), attrs = ['bold']))
print(cl('Profit percentage of the STOCH MACD strategy : {}%'.format(profit_percentage), attrs = ['bold']))

Profit gained from the STOCH MACD strategy by investing $100k in Infosys : 5817.93
Profit percentage of the STOCH MACD strategy : 5%


/tmp/ipykernel_22120/919840583.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  returns = df_ret['returns'][i] * strategy['stoch_macd_position'][i]
/tmp/ipykernel_22120/919840583.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  number_of_stocks = floor(investment_value / df['Close'][0])
